In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generate

> generater

In [ ]:
#| default_exp gen

In [ ]:
#| hide
#| export
import jax.numpy as jnp
import jax.random as jrd
import jax.lax as lax
import jax
from jaxtyping import Array, Float, Int, PRNGKeyArray
from typing import Tuple
from functools import partial

In [ ]:
#| export
@partial(jax.jit, static_argnames=['N', 'T'])
def gen_w(
    key: PRNGKeyArray,
    N: int, # $N$
    T: int, # $T$
    G: Float[Array, "{N} {N}"], # $\boldsymbol\Gamma$
    w0: Float[Array, "{N}"], # $\hat{\mathbf w}_{0/-1}$
) -> Float[Array, "{T} {N}"]: # $\{\hat{\mathbf w}_{t/t}\}_{t=0,\ldots,T-1}$
    keys = jrd.split(key, T)
    def step(wtpre, key):
        wt = jrd.multivariate_normal(key, wtpre, G)
        return wt, wt
    _, W = lax.scan(step, w0, keys, length=T)
    return W

In [ ]:
#| export
@partial(jax.jit, static_argnames=['N', 'T'])
def gen_xy(
    key: PRNGKeyArray,
    N: int, # $N$
    T: int, # $T$
    Sigma: Float[Array, "{N} {N}"], # $\boldsymbol\Sigma$
    W: Float[Array, "{T} {N}"], # $\{\hat{\mathbf w}_{t/t}\}_{t=0,\ldots,T-1}$
    propy1: Float[Array, ""],  # $p(y=1)$
) -> Tuple[Float[Array, "{T} {N}"], Float[Array, "{T}"]]:
    # split key for independent draws
    key_y, key_z = jrd.split(key, 2)

    # Bernoulli draws (returns bool) -> convert to float
    Y = jrd.bernoulli(key_y, p=propy1, shape=(T,)).astype(jnp.float32)  # shape (T,)

    # Cholesky of Sigma (assumes positive-definite). Sigma shape (N,N)
    L = jnp.linalg.cholesky(Sigma)  # lower-triangular, (N,N)

    # Compute per-time means: mu_t = 0.5 * Sigma @ W[t]
    # Vectorized: W @ Sigma.T yields (T,N) where row t is W[t] @ Sigma.T == (Sigma @ W[t])^T
    sign = 2 * Y - 1
    mu = 0.5 * ((W * sign[:, None]) @ Sigma.T)  # shape (T, N)

    # Standard normal samples z_t ~ N(0, I) stacked to shape (T, N)
    z = jrd.normal(key_z, shape=(T, N))

    # Transform: X = mu + L @ z_t  <=>  mu + z @ L.T
    X = mu + (z @ L.T)  # shape (T, N)

    return X, Y

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)